# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

val housing = spark.read.option("header", "true").option("inferSchema", "True").csv("/FileStore/tables/lab1/housing.csv")


housing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code
housing.schema

res1: org.apache.spark.sql.types.StructType = StructType(StructField(longitude,DoubleType,true), StructField(latitude,DoubleType,true), StructField(housing_median_age,DoubleType,true), StructField(total_rooms,DoubleType,true), StructField(total_bedrooms,DoubleType,true), StructField(population,DoubleType,true), StructField(households,DoubleType,true), StructField(median_income,DoubleType,true), StructField(median_house_value,DoubleType,true), StructField(ocean_proximity,StringType,true))

Print the number of records in the dataset.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

housing.count

res2: Long = 20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 5 rows

Print the number of records with population more than 10000.

In [0]:
%scala

// TODO: Replace <FILL IN> with appropriate code

housing.filter($"population" >=10000).count

res4: Long = 23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

housing.select("housing_median_age", "total_rooms", "median_house_value", "population").describe().show()

+-------+------------------+------------------+------------------+------------------+
summary|housing_median_age| total_rooms|median_house_value| population|
+-------+------------------+------------------+------------------+------------------+
 count| 20640| 20640| 20640| 20640|
 mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
 stddev| 12.58555761211163|2181.6152515827944|115395.61587441359| 1132.46212176534|
 min| 1.0| 2.0| 14999.0| 3.0|
 max| 52.0| 39320.0| 500001.0| 35682.0|
+-------+------------------+------------------+------------------+------------------+

Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.sql.functions._

housing.agg(max("housing_median_age"), min("total_rooms"), avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
 52.0| 2.0| 206855.81690891474|
+-----------------------+----------------+-----------------------+

import org.apache.spark.sql.functions._

## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").count().sort(desc("count")).show()

+---------------+-----+
ocean_proximity|count|
+---------------+-----+
 <1H OCEAN| 9136|
 INLAND| 6551|
 NEAR OCEAN| 2658|
 NEAR BAY| 2290|
 ISLAND| 5|
+---------------+-----+

Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [0]:
%scala
housing.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
 -122.25| 37.85| 52.0| 919.0| 213.0| 413.0| 193.0| 4.0368| 269700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 2535.0| 489.0| 1094.0| 514.0| 3.6591| 299200.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3104.0| 687.0| 1157.0| 647.0| 3.12| 241400.0| NEAR BAY|
 -122.26| 37.84| 42.0| 2555.0| 665.0| 1206.0| 595.0| 2.0804| 226700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3549.0| 707.0| 1551.0| 714.0| 3.6912| 261100.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2202.0| 434.0| 910.0| 402.0| 3.2031| 281500.0| NEAR BAY|
 -122.26| 37.85| 52.0| 3503.0| 752.0| 1504.0| 734.0| 3.2705| 241800.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2491.0| 474.0| 1098.0| 468.0| 3.075| 213500.0| NEAR BAY|
 -122.26| 37.84| 52.0| 696.0| 191.0| 345.0| 174.0| 2.6736| 191300.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2643.0| 626.0| 1212.0| 620.0| 1.9167| 159200.0| NEAR BAY|
 -122.26| 37.85| 50.0| 1120.0| 283.0| 697.0| 264.0| 2.125| 140000.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1966.0| 347.0| 793.0| 331.0| 2.775| 152500.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1228.0| 293.0| 648.0| 303.0| 2.1202| 155500.0| NEAR BAY|
 -122.26| 37.84| 50.0| 2239.0| 455.0| 990.0| 419.0| 1.9911| 158700.0| NEAR BAY|
 -122.27| 37.84| 52.0| 1503.0| 298.0| 690.0| 275.0| 2.6033| 162900.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 20 rows

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code
import org.apache.spark.sql.types._
//val df2 = housing.withColumn("avgtmp", $"median_house_value".cast(IntegerType))
//    .drop("median_house_value")
 //   .withColumnRenamed("avgtmp", "median_house_value")

housing.groupBy("ocean_proximity").avg("median_house_value").withColumnRenamed("avg(median_house_value)", "avg_value").show()

+---------------+------------------+
ocean_proximity| avg_value|
+---------------+------------------+
 ISLAND| 380440.0|
 NEAR OCEAN|249433.97742663656|
 NEAR BAY|259212.31179039303|
 <1H OCEAN|240084.28546409807|
 INLAND|124805.39200122119|
+---------------+------------------+

import org.apache.spark.sql.types._

Rewrite the above question in SQL.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) as avg_value FROM df GROUP BY ocean_proximity").show()

+---------------+------------------+
ocean_proximity| avg_value|
+---------------+------------------+
 ISLAND| 380440.0|
 NEAR OCEAN|249433.97742663656|
 NEAR BAY|259212.31179039303|
 <1H OCEAN|240084.28546409807|
 INLAND|124805.39200122119|
+---------------+------------------+

## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler()
  .setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population"))
  .setOutputCol("correlation_features")

val housingAttrs = va.transform(housing)
housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|correlation_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|[41.0,880.0,45260...|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|[21.0,7099.0,3585...|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|[52.0,1467.0,3521...|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|[52.0,1274.0,3413...|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|[52.0,1627.0,3422...|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_aea599ae8488, handleInvalid=error, numInputCols=4
housingAttrs: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "correlation_features").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0 -0.3612622012223152 0.10562341249321029 -0.29624423977353675 
-0.3612622012223152 1.0 0.13415311380656375 0.8571259728659744 
0.10562341249321029 0.13415311380656375 1.0 -0.024649678888894997 
-0.29624423977353675 0.8571259728659744 -0.024649678888894997 1.0 
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0 -0.3612622012223152 0.10562341249321029 -0.29624423977353675
-0.3612622012223152 1.0 0.13415311380656375 0.8571259728659744
0.10562341249321029 0.13415311380656375 1.0 -0.024649678888894997
-0.29624423977353675 0.8571259728659744 -0.024649678888894997 1.0

## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

val housingCol1 = housing.withColumn("rooms_per_household", $"total_rooms"/$"households")
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", $"total_bedrooms"/$"total_rooms")
val housingExtra = housingCol2.withColumn("population_per_household", $"population"/$"households")

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
rooms_per_household| bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
 6.984126984126984|0.14659090909090908| 2.5555555555555554|
 6.238137082601054|0.15579659106916466| 2.109841827768014|
 8.288135593220339|0.12951601908657123| 2.8022598870056497|
 5.8173515981735155|0.18445839874411302| 2.547945205479452|
 6.281853281853282| 0.1720958819913952| 2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows

housingCol1: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]
housingCol2: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
housingExtra: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")

renamedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors.

In [0]:
%scala
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel: String = label
colCat: String = ocean_proximity
colNum: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [0]:
%scala
colNum

res17: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)

In [0]:
%scala
import org.apache.spark.sql.functions._
// TODO: Replace <FILL IN> with appropriate code
var tmp = renamedHousing.select(renamedHousing.columns.map(c => sum(col(c).isNull.cast("int")).alias(c)): _*).select(colNum.map(col): _*).show()


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-------------------+-----------------+------------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|rooms_per_household|bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-------------------+-----------------+------------------------+
 0| 0| 0| 0| 207| 0| 0| 0| 0| 207| 0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-------------------+-----------------+------------------------+

import org.apache.spark.sql.functions._
tmp: Unit = ()

As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median")
  .setInputCols(Array("total_bedrooms", "bedrooms_per_room"))
  .setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))

val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)
  .select("longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "rooms_per_household", "bedrooms_per_room", "population_per_household")

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
total_bedrooms| bedrooms_per_room|
+--------------+-------------------+
 129.0|0.14659090909090908|
 1106.0|0.15579659106916466|
 190.0|0.12951601908657123|
 235.0|0.18445839874411302|
 280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_71541df7a727
imputedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [0]:
%scala
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
var cols = Array("longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "rooms_per_household", "bedrooms_per_room", "population_per_household")
val va = new VectorAssembler()
  .setInputCols(cols)
  .setOutputCol("features")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")
  .setWithStd(true)
  .setWithMean(false)
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.select("scaledFeatures").show(5)

+--------------------+
 scaledFeatures|
+--------------------+
[-61.007269596069...|
[-61.002278409814...|
[-61.012260782324...|
[-61.017251968579...|
[-61.017251968579...|
+--------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
cols: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_4fd49ed3c02e, handleInvalid=error, numInputCols=11
featuredHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_cc83759561f6
scaledHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().<FILL IN>
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

renamedHousing.select("ocean_proximity").distinct.show

+---------------+
ocean_proximity|
+---------------+
 ISLAND|
 NEAR OCEAN|
 NEAR BAY|
 <1H OCEAN|
 INLAND|
+---------------+

### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_proximity_indexed")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income| label|ocean_proximity|rooms_per_household| bedrooms_per_room|population_per_household|ocean_proximity_indexed|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252|452600.0| NEAR BAY| 6.984126984126984|0.14659090909090908| 2.5555555555555554| 3.0|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014|358500.0| NEAR BAY| 6.238137082601054|0.15579659106916466| 2.109841827768014| 3.0|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574|352100.0| NEAR BAY| 8.288135593220339|0.12951601908657123| 2.8022598870056497| 3.0|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431|341300.0| NEAR BAY| 5.8173515981735155|0.18445839874411302| 2.547945205479452| 3.0|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462|342200.0| NEAR BAY| 6.281853281853282| 0.1720958819913952| 2.1814671814671813| 3.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.StringIndexer
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_69469f6c2269
idxHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [0]:
%scala
indexer.fit(renamedHousing).labelsArray

res28: Array[Array[String]] = Array(Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND))

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.OneHotEncoder

val encoder = new OneHotEncoder()
  .setInputCol("ocean_proximity_indexed")
  .setOutputCol("ocean_prox_encoded")
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income| label|ocean_proximity|rooms_per_household| bedrooms_per_room|population_per_household|ocean_proximity_indexed|ocean_prox_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252|452600.0| NEAR BAY| 6.984126984126984|0.14659090909090908| 2.5555555555555554| 3.0| (4,[3],[1.0])|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014|358500.0| NEAR BAY| 6.238137082601054|0.15579659106916466| 2.109841827768014| 3.0| (4,[3],[1.0])|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574|352100.0| NEAR BAY| 8.288135593220339|0.12951601908657123| 2.8022598870056497| 3.0| (4,[3],[1.0])|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431|341300.0| NEAR BAY| 5.8173515981735155|0.18445839874411302| 2.547945205479452| 3.0| (4,[3],[1.0])|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462|342200.0| NEAR BAY| 6.281853281853282| 0.1720958819913952| 2.1814671814671813| 3.0| (4,[3],[1.0])|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.OneHotEncoder
encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_c4365c92df56
ohHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing).drop("features")

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+-----------------------+------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income| label|ocean_proximity|rooms_per_household| bedrooms_per_room|population_per_household| scaledFeatures|ocean_proximity_indexed|ocean_prox_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+-----------------------+------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252|452600.0| NEAR BAY| 6.984126984126984|0.14659090909090908| 2.5555555555555554|[-61.007269596069...| 3.0| (4,[3],[1.0])|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014|358500.0| NEAR BAY| 6.238137082601054|0.15579659106916466| 2.109841827768014|[-61.002278409814...| 3.0| (4,[3],[1.0])|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574|352100.0| NEAR BAY| 8.288135593220339|0.12951601908657123| 2.8022598870056497|[-61.012260782324...| 3.0| (4,[3],[1.0])|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431|341300.0| NEAR BAY| 5.8173515981735155|0.18445839874411302| 2.547945205479452|[-61.017251968579...| 3.0| (4,[3],[1.0])|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462|342200.0| NEAR BAY| 6.281853281853282| 0.1720958819913952| 2.1814671814671813|[-61.017251968579...| 3.0| (4,[3],[1.0])|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+-----------------------+------------------+
only showing top 5 rows

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
numPipeline: org.apache.spark.ml.Pipeline = pipeline_22caaa728db1
catPipeline: org.apache.spark.ml.Pipeline = pipeline_586f17ce55bf
pipeline: org.apache.spark.ml.Pipeline = pipeline_90267414e756
newHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 14 more fields]

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

val va2 = new VectorAssembler()
  .setInputCols(Array("scaledFeatures", "ocean_prox_encoded"))
  .setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
 features| label|
+--------------------+--------+
[-61.007269596069...|452600.0|
[-61.002278409814...|358500.0|
[-61.012260782324...|352100.0|
[-61.017251968579...|341300.0|
[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows

va2: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_447621776157, handleInvalid=error, numInputCols=2
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: double]

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]
res62: Long = 4111

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression().setMaxIter(10).setElasticNetParam(0.1).setRegParam(0.5)
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"R-squared: ${trainingSummary.r2}")

Coefficients: [-18072.82798239788,-15001.831141742388,14523.97307818654,6510.725804081123,18641.16881061436,-37444.461608991165,18162.191450056995,82693.66653897235,2993.474372421311,18881.001876267386,-90.81415068561454,16998.900699994992,-36841.6507135625,28362.389732228723,17386.297007337616], Intercept: -909423.0712209494
RMSE: 68256.17694962754
R-squared: 0.651661392460911
import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_2d8613e20454
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = LinearRegressionModel: uid=linReg_2d8613e20454, numFeatures=15
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@6da9e7ca

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setPredictionCol("prediction")
  .setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
 prediction| label| features|
+------------------+--------+--------------------+
214144.81923173636| 94600.0|[-62.065401082150...|
181992.28784159466|103600.0|[-62.040445150874...|
222163.02506525663|106700.0|[-62.005506847089...|
 217628.7971001739|128900.0|[-61.975559729558...|
 152837.1718887746| 68300.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 70736.70392724255
import org.apache.spark.ml.evaluation.RegressionEvaluator
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_529817216354, metricName=rmse, throughOrigin=false
rmse: Double = 70736.70392724255

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dtr = new DecisionTreeRegressor()
  .setLabelCol("label")
  .setFeaturesCol("features")
// train the model
val dtModel = dtr.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setPredictionCol("prediction")
  .setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
 prediction| label| features|
+------------------+--------+--------------------+
 166668.141723356| 94600.0|[-62.065401082150...|
 166668.141723356|103600.0|[-62.040445150874...|
 166668.141723356|106700.0|[-62.005506847089...|
225471.38360175694|128900.0|[-61.975559729558...|
131986.66666666666| 68300.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 68805.05776623574
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
dtr: org.apache.spark.ml.regression.DecisionTreeRegressor = dtr_cd55ef6c4fb5
dtModel: org.apache.spark.ml.regression.DecisionTreeRegressionModel = DecisionTreeRegressionModel: uid=dtr_cd55ef6c4fb5, depth=5, numNodes=63, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_c1ab9f2dfcd1, metricName=rmse, throughOrigin=false
rmse: Double = 68805.05776623574

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor()
  .setLabelCol("label")
  .setFeaturesCol("features")
// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setPredictionCol("prediction")
  .setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
 prediction| label| features|
+------------------+--------+--------------------+
225291.25984759885| 94600.0|[-62.065401082150...|
170707.01610683525|103600.0|[-62.040445150874...|
217920.82672573806|106700.0|[-62.005506847089...|
239978.98921641853|128900.0|[-61.975559729558...|
 182416.6447512183| 68300.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67226.3826319314
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_f74e4657d0a5
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel: uid=rfr_f74e4657d0a5, numTrees=20, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_97b5b99a9f8a, metricName=rmse, throughOrigin=false
rmse: Double = 67226.3826319314

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [0]:
%scala
trainSet.show(1)

+--------------------+-------+
 features| label|
+--------------------+-------+
[-62.040445150874...|85800.0|
+--------------------+-------+
only showing top 1 row

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor()
  .setLabelCol("label")
  .setFeaturesCol("features")
// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setPredictionCol("prediction")
  .setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
 prediction| label| features|
+------------------+--------+--------------------+
 154172.2986563275| 94600.0|[-62.065401082150...|
110296.36987270221|103600.0|[-62.040445150874...|
144209.55530902446|106700.0|[-62.005506847089...|
123196.33176988826|128900.0|[-61.975559729558...|
 74594.01349042215| 68300.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 57477.9022016309
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
gb: org.apache.spark.ml.regression.GBTRegressor = gbtr_d4d0e62c6394
gbModel: org.apache.spark.ml.regression.GBTRegressionModel = GBTRegressionModel: uid=gbtr_d4d0e62c6394, numTrees=20, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_18a22e86f3dd, metricName=rmse, throughOrigin=false
rmse: Double = 57477.9022016309

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [0]:
%scala
cvModel.

res103: String =
estimator: estimator for selection (current: linReg_2d8613e20454)
estimatorParamMaps: param maps for the estimator (current: [Lorg.apache.spark.ml.param.ParamMap;@7d6c9ece)
evaluator: evaluator used to select hyper-parameters that maximize the validated metric (current: RegressionEvaluator: uid=regEval_c994bc8ce01b, metricName=mse, throughOrigin=false)
numFolds: number of folds for cross validation (>= 2) (default: 3, current: 3)
seed: random seed (default: -1191137437)

In [0]:
%scala
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
  .addGrid(rf.numTrees, Array(1, 5, 10))
  .addGrid(rf.maxDepth, Array(5, 10 ,15))
  .build()

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("mse")
val cv = new CrossValidator()
  .setEstimator(lr)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)
val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
 prediction| label| features|
+------------------+--------+--------------------+
214144.81923173636| 94600.0|[-62.065401082150...|
181992.28784159466|103600.0|[-62.040445150874...|
222163.02506525663|106700.0|[-62.005506847089...|
 217628.7971001739|128900.0|[-61.975559729558...|
 152837.1718887746| 68300.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 5.003681282490371E9
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfr_f74e4657d0a5-maxDepth: 5,
	rfr_f74e4657d0a5-numTrees: 1
}, {
	rfr_f74e4657d0a5-maxDepth: 5,
	rfr_f74e4657d0a5-numTrees: 5
}, {
	rfr_f74e4657d0a5-maxDepth: 5,
	rfr_f74e4657d0a5-numTrees: 10
}, {
	rfr_f74e4657d0a5-maxDepth: 10,
	rfr_f74e4657d0a5-numTrees: 1
}, {
	rfr_f74e4657d0a5-maxDepth: 10,
	rfr_f74e4657d0a5-numTrees: 5
}, {
	rfr_f74e4657d0a5-maxDepth: 10,
	rfr_f74e4657d0a5-numTrees: 10
}, {
	rfr_f74e4657d0a5-maxDepth: 15,
	rfr_f74e4657d0a5-numTrees: 1
}, {
	rfr_f74e4657d0a5-maxDepth: 15,
	rfr_f74e4657d0a5-numTrees: 5
}, {
	rfr_f74e4657d0a5-maxDepth: 15,
	rfr_f74e4657d0a5-numTrees: 10
})
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_c994bc8ce01b, metricName=mse, throughOrigin=false
cv: org.apache.spark.ml.tuning.CrossValidator = cv_e0d142d4c281
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_e0d142d4c281, bestModel=LinearRegressionModel: uid=linReg_2d8613e20454, numFeatures=15, numFolds=3
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
rmse: Double = 5.003681282490371E9

---
# 7. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?